# 各種設定

In [ ]:
!pip install timm

In [ ]:
import os
import sys
import math
import random
import time
import warnings


import numpy as np
import pandas as pd

from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


from tqdm.notebook import tqdm
import torchvision.models as models
import timm

import pytorch_lightning as pl
from pytorch_lightning import LightningModule
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import Trainer


def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

# seed = 2999
seed = 2021
seed_torch(seed)

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# モデルの定義

In [ ]:
import torch.nn.functional as F
import torchvision.models as models


class Create_Model(nn.Module):
    def __init__(self):
        super(Create_Model,self).__init__()
        self.swin = timm.create_model('swin_large_patch4_window7_224',pretrained=True, num_classes=0, in_chans=3)
        num_features = self.swin.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, 100),
            nn.Dropout(0.5), nn.Linear(100, 1)
        )
        
    def forward(self,x):
        x = self.swin(x)
        x = self.fc(x)
        return x

# データローダーの定義

In [ ]:
train_df = pd.read_csv('/kaggle/input/petfinder-train-add-class/train_add_class.csv')

In [ ]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image 
import torchvision.transforms as T
from tqdm import tqdm
from torchvision.io import read_image

class CustomDataSet(Dataset):
    def __init__(self,df,key = "train",cls_type = 'dog'):
        self.df = df[df['class'] == cls_type]
        self.img_list =  self.df['Id'].to_list()
        self.table_list = self.df.loc[:,['Subject Focus','Eyes','Face','Near','Action','Accessory','Group','Collage','Human','Occlusion','Info','Blur']]
        #self.table_list = self.df[self.df.columns[1:13]]
        self.label_list = self.df['Pawpularity'].to_list()
        self.preprocess = {"train":nn.Sequential(
                                T.Resize(256),
                                T.CenterCrop(224),
                                T.ConvertImageDtype(torch.float32),
                                T.RandomHorizontalFlip(),
                                T.RandomVerticalFlip(),
                                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                                T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                                T.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                            ).to(torch.device('cuda'))
                           ,"val":nn.Sequential(
                                T.Resize(256),
                                T.CenterCrop(224),
                                T.ConvertImageDtype(torch.float32),
                                T.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                            ).to(torch.device('cuda'))
                                              
                          }
    
        self.key = key
        self.cls_type = cls_type
    def __getitem__(self,idx):
        path = "/kaggle/input/petfinder-pawpularity-score/train/"+self.img_list[idx]+'.jpg'
        img = read_image(path)
        img = self.preprocess[self.key](img)
        tbl = torch.tensor(list(self.table_list.iloc[idx])).to(torch.float32)
        label = torch.tensor(self.label_list[idx]/100).to(torch.float32)
        return img,tbl,label
    def __len__(self):
        return len(self.img_list)

In [ ]:
# FOLD_NUM = 7
FOLD_NUM = 5

# 実験内容

In [ ]:
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from timm.scheduler import CosineLRScheduler
import torch.optim 
import numpy as np

kf = KFold(n_splits=FOLD_NUM, random_state=None, shuffle=False)

BATCH_SIZE = 8
EPOCH = 100
STOP_NUM = 5
criterion_mse = nn.MSELoss()
criterion_bce = nn.BCEWithLogitsLoss()

best_score_list = {'cat':[] ,'dog':[]}
for k,(train_index, test_index) in enumerate(kf.split(train_df['Id'])):
    for cls_type in ['cat' , 'dog']:
        
        print(f"========={k+1}-FOLD-{cls_type}=========")
        model = Create_Model().to(DEVICE)

        train_dataset = CustomDataSet(train_df.loc[train_index],cls_type = cls_type)
        val_dataset = CustomDataSet(train_df.loc[test_index],key = 'val',cls_type = cls_type)
        train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE,shuffle = True,num_workers = 2,pin_memory=True)
        val_loader = DataLoader(val_dataset,batch_size = BATCH_SIZE,shuffle = False,num_workers = 2,pin_memory=True)

        optimizer = optim.Adam(params = model.parameters(),lr = 1e-6)
        #cheduler = scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000)

        best_loss = 99999
        stop_counter = 0
        scheduler = CosineLRScheduler(optimizer, t_initial=50, lr_min=1e-7, warmup_t=20, warmup_lr_init=1e-8, warmup_prefix=True)
        for e in range(EPOCH):
            model.train()
            train_total_loss = 0
            train_score = 0
            counter = 0        

            for n,(img,tbl,label) in enumerate(train_loader):

                img = img.to(DEVICE)
                tbl = tbl.to(DEVICE)
                label = label.to(DEVICE)
                optimizer.zero_grad()

                output = model(img)[:, 0]

                loss = criterion_bce(output, label)
                #loss = criterion_mse(output,label)
                score = criterion_mse(torch.sigmoid(output)*100 , label*100).detach()

                train_total_loss = (train_total_loss * n + loss.detach().item())/(n+1)
                train_score =  (train_score * counter + score.item()*output.shape[0])/(counter + output.shape[0])

                counter += output.shape[0]
                loss.backward()
                optimizer.step()
                print('\rTRAIN EPOCH[{:03}/{:03}] ITR[{:04}/{:04}] LOSS:{:.5} SCORE:{:.5}'.format(e+1,EPOCH,n+1,len(train_loader),train_total_loss,np.sqrt(train_score)),end = "")
            scheduler.step(e+1)
            print()


            val_total_loss = 0
            val_score = 0
            counter = 0


            model.eval()
            with torch.no_grad():
                for n,(img,tbl,label) in enumerate(val_loader):
                    img = img.to(DEVICE)
                    tbl = tbl.to(DEVICE)
                    label = label.to(DEVICE)

                    output = model(img)[:, 0]

                    loss = criterion_bce(output, label)
                    score = criterion_mse(torch.sigmoid(output)*100 , label*100).detach()


                    val_total_loss = (val_total_loss * n + loss.item())/(n+1)
                    val_score =  (val_score * counter + score.item()*output.shape[0])/(counter + output.shape[0])

                    counter += output.shape[0]
                    print('\rVAL   EPOCH[{:03}/{:03}] ITR[{:04}/{:04}] LOSS:{:.5} SCORE:{:.5}'.format(e+1,EPOCH,n+1,len(val_loader),val_total_loss,np.sqrt(val_score)),end = "")
            print()
            if(best_loss > val_total_loss):
                best_loss = val_total_loss
                model_path = f'{cls_type}-{k+1}-fold.pth'
                torch.save(model.state_dict(), model_path)
                stop_counter = 0
                best_score = np.sqrt(val_score)
            else:
                stop_counter += 1
            if stop_counter >= STOP_NUM:
                break
        best_score_list[cls_type].append(best_score)

In [ ]:
best_score_list

In [ ]:
for key in ['cat','dog']:
    print(sum(best_score_list[key]) / len(best_score_list[key]))